# PySpark Cookbook

### Tomasz Drabas, Denny Lee
#### Version: 0.1
#### Date: 1/15/2018

# Some data

In [1]:
dirty_data = spark.createDataFrame([
          (1,'Porsche','Boxster S','Turbo',2.5,4,22,None)
        , (2,'Aston Martin','Vanquish, S and Zagato','Aspirated',6.0,12,16,None)
        , (3,'Porsche','911 Carrera 4S Cabriolet','Turbo',3.0,6,24,None)
        , (3,'General Motors','SPARK ACTIV','Aspirated',1.4,None,32,None)
        , (5,'BMW','COOPER S HARDTOP 2 DOOR','Turbo',2.0,4,26,None)
        , (6,'BMW','330i','Turbo',2.0,None,27,None)
        , (7,'BMW','440i Coupe','Turbo',3.0,6,23,None)
        , (8,'BMW','440i Coupe','Turbo',3.0,6,23,None)
        , (9,'Mercedes-Benz',None,None,None,None,27,None)
        , (10,'Mercedes-Benz','CLS 550','Turbo',4.7,8,21,79231)
        , (11,'Volkswagen','GTI','Turbo',2.0,4,None,None)
        , (12,'Ford Motor Company','FUSION AWD','Turbo',2.7,6,20,None)
        , (13,'Nissan','Q50 AWD RED SPORT','Turbo',3.0,6,22,None)
        , (14,'Nissan','Q70 AWD','Aspirated',5.6,8,18,None)
        , (15,'Kia','Stinger RWD','Turbo',2.0,4,25,None)
        , (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None)
        , (16,'Toyota','CAMRY HYBRID LE','Aspirated',2.5,4,46,None)
        , (18,'FCA US LLC','300','Aspirated',3.6,6,23,None)
        , (19,'Hyundai','G80 AWD','Turbo',3.3,6,20,None)
        , (20,'Hyundai','G80 AWD','Turbo',3.3,6,20,None)
        , (21,'BMW','X5 M','Turbo',4.4,8,18,121231)
        , (22,'General Motors','K1500 SUBURBAN 4WD','Aspirated',5.3,8,18,None)
    ], ['Id','Manufacturer','Model','EngineType','Displacement','Cylinders','FuelEconomy','MSRP']
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,None,pyspark,idle,,,✔


SparkSession available as 'spark'.


# Handling duplicates

### Exact duplicates

In [2]:
# do we have any rows that are duplicated?
dirty_data.count(), dirty_data.distinct().count()

(22, 21)

In [3]:
# what row is duplicated?
(
    dirty_data
    .groupby(dirty_data.columns)
    .count()
    .filter('count > 1')
    .show()
)

+---+------------+---------------+----------+------------+---------+-----------+----+-----+
| Id|Manufacturer|          Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|count|
+---+------------+---------------+----------+------------+---------+-----------+----+-----+
| 16|      Toyota|CAMRY HYBRID LE| Aspirated|         2.5|        4|         46|null|    2|
+---+------------+---------------+----------+------------+---------+-----------+----+-----+

In [4]:
# remove the duplicated rows
full_removed = dirty_data.dropDuplicates()
full_removed.count()

21

### Only ID differs

In [5]:
# count of rows
no_ids = (
    full_removed
    .select([col for col in full_removed.columns if col != 'Id'])
)

no_ids.count(), no_ids.distinct().count()

(21, 19)

In [6]:
# what row is duplicated?
(
    full_removed
    .groupby([col for col in full_removed.columns if col != 'Id'])
    .count()
    .filter('count > 1')
    .show()
)

+------------+----------+----------+------------+---------+-----------+----+-----+
|Manufacturer|     Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|count|
+------------+----------+----------+------------+---------+-----------+----+-----+
|         BMW|440i Coupe|     Turbo|         3.0|        6|         23|null|    2|
|     Hyundai|   G80 AWD|     Turbo|         3.3|        6|         20|null|    2|
+------------+----------+----------+------------+---------+-----------+----+-----+

In [7]:
# remove the duplicated record
id_removed = full_removed.dropDuplicates(
    subset = [col for col in full_removed.columns if col != 'Id']
)

id_removed.count()

19

### Duplicated IDs

In [8]:
# are there any duplicated IDs?
import pyspark.sql.functions as fn

id_removed.agg(
      fn.count('Id').alias('CountOfIDs')
    , fn.countDistinct('Id').alias('CountOfDistinctIDs')
).show()

+----------+------------------+
|CountOfIDs|CountOfDistinctIDs|
+----------+------------------+
|        19|                18|
+----------+------------------+

In [9]:
# what's duplicated?
(
    id_removed
    .groupby('Id')
    .count()
    .filter('count > 1')
    .show()
)

+---+-----+
| Id|count|
+---+-----+
|  3|    2|
+---+-----+

In [10]:
new_id = (
    id_removed
    .withColumn('NewId', fn.monotonically_increasing_id())
    .select([fn.col('NewId').alias('Id')] + [col for col in id_removed.columns if col != 'Id'])
)

new_id.show()

+-------------+------------------+--------------------+----------+------------+---------+-----------+------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|FuelEconomy|  MSRP|
+-------------+------------------+--------------------+----------+------------+---------+-----------+------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|     null|         32|  null|
|  17179869184|    General Motors|  K1500 SUBURBAN 4WD| Aspirated|         5.3|        8|         18|  null|
|  94489280512|      Aston Martin|Vanquish, S and Z...| Aspirated|         6.0|       12|         16|  null|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|         21| 79231|
| 197568495616|     Mercedes-Benz|                null|      null|        null|     null|         27|  null|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        6|         20|  null|
| 438086664192|    

# Handling missing observations

### Missing observations per row

In [20]:
(
    spark.createDataFrame(
        new_id.rdd.map(
           lambda row: (
                 row['Id']
               , sum([c == None for c in row])
           )
        ).collect()
        ,['Id', 'CountMissing']
    )
    .filter('CountMissing > 1')
    .orderBy('CountMissing', ascending=False)
    .show()
)

+------------+------------+
|          Id|CountMissing|
+------------+------------+
|197568495616|           5|
|  8589934592|           2|
|919123001344|           2|
|721554505728|           2|
+------------+------------+

In [12]:
new_id.where('Id == 197568495616').show()

+------------+-------------+-----+----------+------------+---------+-----------+----+
|          Id| Manufacturer|Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|
+------------+-------------+-----+----------+------------+---------+-----------+----+
|197568495616|Mercedes-Benz| null|      null|        null|     null|         27|null|
+------------+-------------+-----+----------+------------+---------+-----------+----+

In [13]:
merc_out = new_id.dropna(thresh=6)
new_id.count(), merc_out.count()

(19, 18)

In [14]:
merc_out.where('Id == 197568495616').show()

+---+------------+-----+----------+------------+---------+-----------+----+
| Id|Manufacturer|Model|EngineType|Displacement|Cylinders|FuelEconomy|MSRP|
+---+------------+-----+----------+------------+---------+-----------+----+
+---+------------+-----+----------+------------+---------+-----------+----+

### Missing observations per column

In [15]:
for k, v in sorted(
    merc_out.agg(*[
               (1 - (fn.count(c) / fn.count('*')))
                    .alias(c + '_miss')
               for c in new_id.columns
           ])
        .collect()[0]
        .asDict()
        .items()
    , key=lambda el: el[1]
    , reverse=True
):
    print(k, v)

MSRP_miss 0.8888888888888888
Cylinders_miss 0.11111111111111116
FuelEconomy_miss 0.05555555555555558
EngineType_miss 0.0
Manufacturer_miss 0.0
Id_miss 0.0
Model_miss 0.0
Displacement_miss 0.0

In [16]:
no_MSRP = merc_out.select([col for col in new_id.columns if col != 'MSRP'])
no_MSRP.show()

+-------------+------------------+--------------------+----------+------------+---------+-----------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|FuelEconomy|
+-------------+------------------+--------------------+----------+------------+---------+-----------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|     null|         32|
|  17179869184|    General Motors|  K1500 SUBURBAN 4WD| Aspirated|         5.3|        8|         18|
|  94489280512|      Aston Martin|Vanquish, S and Z...| Aspirated|         6.0|       12|         16|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|         21|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        6|         20|
| 438086664192|               BMW|COOPER S HARDTOP ...|     Turbo|         2.0|        4|         26|
| 721554505728|        Volkswagen|                 GTI|     Turbo|         2.0|   

### Sparse missing observations

In [52]:
multipliers = (
    no_MSRP
    .agg(
          fn.mean(fn.col('FuelEconomy') / (fn.col('Displacement') * fn.col('Cylinders'))).alias('FuelEconomy')
        , fn.mean(fn.col('Cylinders')   / fn.col('Displacement')).alias('Cylinders')
    )
).toPandas().to_dict('records')[0]

multipliers

# data = no_MSRP.select('FuelEconomy', 'Displacement', 'Cylinders').toPandas()
# data_train = data.dropna()

# from sklearn.linear_model import LinearRegression

# regressor_fuel = LinearRegression()
# fuel_lineModel      = regressor_fuel.fit(data_train[['Displacement', 'Cylinders']], data_train['FuelEconomy'])

# regressor_cylinders = LinearRegression()
# cylinders_lineModel = regressor_cylinders.fit(data_train[['Displacement']], data_train['Cylinders'])

# models = {
#       'FuelEconomy': {'coeff': list(fuel_lineModel.coef_),      'intercept': fuel_lineModel.intercept_}
#     , 'Cylinders':   {'coeff': list(cylinders_lineModel.coef_), 'intercept': cylinders_lineModel.intercept_}
# }
# # fuel_lineModel.coef_, fuel_lineModel.intercept_, cylinders_lineModel.coef_, cylinders_lineModel.intercept_
# models

{'FuelEconomy': 1.4957485048359975, 'Cylinders': 1.8353365984789107}

In [54]:
imputed = (
    no_MSRP
#     .withColumn('Cylinders_imp',   
#                 (
#                     fn.lit(models['Cylinders']['intercept']) + 
#                     fn.lit(models['Cylinders']['coeff'][0]) * fn.col('Displacement')
#                 ).cast('integer')
#     )
#     .withColumn('FuelEconomy_imp', 
#                 fn.lit(models['FuelEconomy']['intercept']) +
#                 fn.lit(models['FuelEconomy']['coeff'][0]) * fn.col('Displacement') +
#                 fn.lit(models['FuelEconomy']['coeff'][1]) * fn.col('Cylinders')
#     )
    .withColumn('FuelEconomy', fn.col('FuelEconomy')   / fn.col('Displacement') / fn.col('Cylinders'))
    .withColumn('Cylinders',   fn.col('Cylinders')   / fn.col('Displacement'))
    .fillna(multipliers)
    .withColumn('Cylinders',   (fn.col('Cylinders')   * fn.col('Displacement')).cast('integer'))
    .withColumn('FuelEconomy', fn.col('FuelEconomy') * fn.col('Displacement') * fn.col('Cylinders'))
)

imputed.show()

+-------------+------------------+--------------------+----------+------------+---------+------------------+
|           Id|      Manufacturer|               Model|EngineType|Displacement|Cylinders|       FuelEconomy|
+-------------+------------------+--------------------+----------+------------+---------+------------------+
|   8589934592|    General Motors|         SPARK ACTIV| Aspirated|         1.4|        2|    4.188095813552|
|  17179869184|    General Motors|  K1500 SUBURBAN 4WD| Aspirated|         5.3|        8|              18.0|
|  94489280512|      Aston Martin|Vanquish, S and Z...| Aspirated|         6.0|       12|              16.0|
| 188978561024|     Mercedes-Benz|             CLS 550|     Turbo|         4.7|        8|              21.0|
| 206158430208|Ford Motor Company|          FUSION AWD|     Turbo|         2.7|        5|16.666666666666668|
| 438086664192|               BMW|COOPER S HARDTOP ...|     Turbo|         2.0|        4|              26.0|
| 721554505728|    

# Handling outliers

In [67]:
features = ['Displacement', 'Cylinders', 'FuelEconomy']
quantiles = [0.25, 0.75]

cut_off_points = []

for feature in features:
    quants = imputed.approxQuantile(feature, quantiles, 0.05)
    
    IQR = quants[1] - quants[0]
    cut_off_points.append((feature, [
        quants[0] - 1.5 * IQR,
        quants[1] + 1.5 * IQR,
    ]))
    
cut_off_points = dict(cut_off_points)

outliers = imputed.select(*['id'] + [
       (
           (imputed[f] < cut_off_points[f][0]) |
           (imputed[f] > cut_off_points[f][1])
       ).alias(f + '_o') for f in features
  ])
outliers.show()

+-------------+--------------+-----------+-------------+
|           id|Displacement_o|Cylinders_o|FuelEconomy_o|
+-------------+--------------+-----------+-------------+
|   8589934592|         false|      false|         true|
|  17179869184|         false|      false|        false|
|  94489280512|         false|      false|        false|
| 188978561024|         false|      false|        false|
| 206158430208|         false|      false|        false|
| 438086664192|         false|      false|        false|
| 721554505728|         false|      false|        false|
| 764504178688|         false|      false|        false|
| 919123001344|         false|      false|        false|
| 944892805120|         false|      false|        false|
| 970662608896|         false|      false|        false|
|1030792151040|         false|      false|        false|
|1039382085632|         false|      false|        false|
|1116691496960|         false|      false|        false|
|1211180777472|         false| 

In [69]:
with_outliers = imputed.join(outliers, on='Id')
with_outliers.filter('FuelEconomy_o').select('Id', 'FuelEconomy').show()

+-------------+--------------+
|           Id|   FuelEconomy|
+-------------+--------------+
|   8589934592|4.188095813552|
|1614907703296|          46.0|
+-------------+--------------+

# Exploring descriptive statistics

# Computing correlations

# Drawing histograms

# Visualizing interactions between features